In [3]:
from sympy import *
init_printing(use_unicode=True)


In [4]:
y = symbols("y", cls=Function)
x = symbols("x")


ZAD1

In [5]:
def RD(expr, war):
    return dsolve(y(x).diff(x)-expr,y(x),ics=war)
#1A
expr1 = 2*y(x)+E**x-x
x0 = 0
y0 = 1/4
display(y(x).diff(x)-expr1,{y(x0):y0})
RD(expr1, {y(x0):y0})

In [6]:
#1B
expr2 = x-1+(x+1)*y(x)
x0 = 0
y0 = 0
display(y(x).diff(x)-expr2,{y(x0):y0})
RD(expr2, {y(x0):y0})
sol=RD(expr2, {y(x0):y0})
display(sol)

Brak możliwości wyznaczenia dokładnego rozwiązania zagadnienia - do tabeli wpisano przybliżone wartości używając evalf()

ZAD 2

In [7]:
def RA(expr, x0, y0, n):
    #y0
    tabY = [y0]
    #y1..n
    for i in range(1,n+1):
        yi = expr.diff(x,i-1)
        for j in range(i):
             yi = yi.subs(y(x).diff(x,i-1-j), tabY[i-1-j])
        yi = yi.subs(x,x0)
        tabY.append(yi)
    #składanie szeregu
    taylor = 0
    for i in range(n+1):
        taylor+=tabY[i]/factorial(i)*(x-x0)**i
    return taylor

RA(2*x**2*y(x)+E**x,2,1,4)

ZAD3

In [8]:
def RN(expr, x0, y0, n, h):
    #rekurencja
    if n==0:
        return {x0:y0}
    before_data = RN(expr, x0, y0, n-1, h)

    #wyznaczanie x - round bo float 0.2000000001
    x_accuracy = min(len(str(h))+1, 6)
    xn = round(x0+n*h,x_accuracy)
    x_before = round(xn-h,x_accuracy)
    y_before = before_data[x_before]

    #obliczenie ze wzoru
    before_data[xn] = y_before + h * expr.subs(y(x),y_before).subs(x,x_before)
    return before_data

RN(x+2+(x-2)*y(x),0,1,4,0.2)    

In [9]:
import pandas as pd
def get_data(expr, x0, y0, n):
    war = {y(x0):y0}
    rd = RD(expr, war)
    ra = RA(expr, x0, y0, n)
    rn = RN(expr, x0, y0, n, 0.2)
    tab = {}
    values = [0, 0.2, 0.4, 0.6, 0.8, 1]
    for i in range(len(values)):
        v = values[i]
        column = []
        column.append(rd.rhs.subs(x,v).evalf(8))
        column.append(ra.subs(x,v).evalf(8))
        column.append(format(rn[v], '.'+str(8)+'f'))
        tab["x"+str(i)+" = " + str(values[i])] = column

    indexes = ["RD", "RA", "RN"]
    tab_df = pd.DataFrame(tab,
                        index=indexes)
    return tab_df

In [10]:
data = get_data(expr1, 0, 1/4, 5)
data.columns.name = "(a)"
display(data)

(a),x0 = 0,x1 = 0.2,x2 = 0.4,x3 = 0.6,x4 = 0.8,x5 = 1
RD,0.25000000,0.62042194,1.1837162,2.0479981,3.3774915,5.4207743
RA,0.25000000,0.62041600,1.1833120,2.0430880,3.3479840,5.3000000
RN,0.25000000,0.55000000,0.97428055,1.58235771,2.45972456,3.72872256


In [11]:
data = get_data(expr2, 0, 0, 5)
data.columns.name = "(b)"
display(data)

(b),x0 = 0,x1 = 0.2,x2 = 0.4,x3 = 0.6,x4 = 0.8,x5 = 1
RD,0,-0.20282866,-0.42446408,-0.69114188,-1.0440676,-1.5526761
RA,0,-0.20282667,-0.42432000,-0.68928000,-1.0321067,-1.5000000
RN,0.00000000,-0.20000000,-0.40800000,-0.64224000,-0.92775680,-1.30174925
